### Linear Regression to predict stay length from MIMIC-III

In [3]:
import argparse
from argparse import ArgumentParser
from datetime import datetime
import json
import logging
from pathlib import Path
import shutil
from typing import Dict, List, Tuple, Type

import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from scipy import sparse
import torch
from torch import nn
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from torchmetrics import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, R2Score


class LinearRegressionModel(nn.Module):
    """
    Linear regression model
    """

    def __init__(self, num_features):
        """
        # Parameters
        num_features : `int`, required.
            Number of the features.
        # Returns
            `None`
        """
        super().__init__()
        # Hw-TODO: Add a linear layer to weight the features.
        self.linear = nn.Linear(in_features=num_features, out_features=1, bias=True)
        self.dropout = nn.Dropout(0.6)

    def forward(self, features):
        """
        Returns the logits of the model given features. 
        Note that model predictions should be either 0 or 1 based on a threshold.
        # Parameters
        features : `torch.FloatTensor`, required.
            The tensor of features with the shape (batch_size, num_of_features)
        # Returns
        probs : `torch.FloatTensor`, required.
            The tensor of output values with the shape (batch_size, 1) or (batch_size,)
        """
        # Hw-TODO: Use `self.linear` you created in `__init__`
        #          and appropriate nonlinearity/activation-function to compute
        #          and return the probabilities of belonging to a class in the logistic regression.
        out = self.linear(features)
        #out = self.dropout(out)
        #probs = torch.sigmoid(out)
        return out

In [4]:
class RegressionModule(pl.LightningModule):

    def __init__(self, **kwargs):
        super().__init__()

        # Save arguments to `hparams` attribute, see the doc [here](https://pytorch-lightning.readthedocs.io/en/latest/common/hyperparameters.html).
        self.save_hyperparameters()
        data_dir = Path(self.hparams.data_dir)
        #self.hparams.vocab = json.load(
        #    open(data_dir.joinpath(self.hparams.vocab_filename)))
        #self.hparams.vocab_size = len(self.hparams.vocab)

        self.model = self.get_model()
        self.step_count = 0
        #self.accuracy = Accuracy()
        #self.pr = Precision(threshold=0.5,average='macro',num_classes=1,multiclass=False)
        self.mse = MeanSquaredError()
        self.mape = MeanAbsolutePercentageError()
        self.mae = MeanAbsoluteError()
        self.r2 = R2Score() 

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

    def training_step(self, batch, batch_idx):
        input = self.batch2input(batch)
        labels = self.batch2labels(batch)
        probs = self(**input)
        probs = probs.squeeze()
        # Hw-TODO: Given probs in shape (batch_size,)
        #          and labels of the same shape,
        #          compute the binary cross entropy loss.
        loss = nn.functional.mse_loss(probs, labels)

        self.log('train_loss', loss, prog_bar=True)
        self.log('train_mse', self.mse(probs, labels.int()), prog_bar=True)
        self.log('train_rmse', self.mape(probs, labels.int()))
        self.log('train_mape', self.mape(probs, labels.int()))
        self.log('train_mae', self.mae(probs, labels.int()))
        self.log('train_r2', self.r2(probs, labels.int()))
        output_dict = {'loss': loss}
        return output_dict

    def validation_step(self, batch, batch_idx):
        input = self.batch2input(batch)
        labels = self.batch2labels(batch)
        probs = self(**input)
        probs = probs.squeeze()

        # Hw-TODO: Given probs in shape (batch_size,)
        #          and labels of the same shape,
        #          compute the binary cross entropy loss.
        loss = nn.functional.mse_loss(probs, labels)

        self.log('val_loss', loss)
        self.log('val_mse', self.mse(probs, labels.int()), prog_bar=True)
        self.log('val_mape', self.mape(probs, labels.int()))
        self.log('val_mae', self.mae(probs, labels.int()))
        self.log('val_r2', self.r2(probs, labels.int()))

    def test_step(self, batch, batch_idx):
        input = self.batch2input(batch)
        labels = self.batch2labels(batch)
        probs = self(**input)
        probs = probs.squeeze()

        # Hw-TODO: Given probs in shape (batch_size,)
        #          and labels of the same shape,
        #          compute the binary cross entropy loss.
        loss = nn.functional.mse_loss(probs, labels)

        self.log('test_loss', loss)
        self.log('test_mse', self.mse(probs, labels.int()), prog_bar=True)
        self.log('test_mape', self.mape(probs, labels.int()))
        self.log('test_mae', self.mae(probs, labels.int()))
        self.log('test_r2', self.r2(probs, labels.int()))

    def configure_optimizers(self):
        if self.hparams.optimizer == 'sgd':
            optimizer = torch.optim.SGD(self.model.parameters(),
                                        lr=self.hparams.learning_rate)
        elif self.hparams.optimizer == 'adam':
            if self.hparams.l2_regularization:
                optimizer = torch.optim.Adam(self.model.parameters(),
                                         lr=self.hparams.learning_rate,
                                         weight_decay=1e-5)
            else:
                optimizer = torch.optim.Adam(self.model.parameters(),
                                         lr=self.hparams.learning_rate)
        else:
            raise NotImplementedError
        return optimizer

    def train_dataloader(self):
        return self.get_dataloader('train', self.hparams.train_batch_size, shuffle=True)

    def val_dataloader(self):
        return self.get_dataloader('dev', self.hparams.eval_batch_size, shuffle=False)

    def test_dataloader(self):
        return self.get_dataloader('test', self.hparams.eval_batch_size, shuffle=False)

    def get_model(self) -> nn.Module:
        # To be overridden by inherited classes.
        raise NotImplementedError

    def batch2input(self, batch: Tuple[torch.Tensor]) -> Dict[str, torch.Tensor]:
        # To be overridden by inherited classes.
        raise NotImplementedError

    def batch2labels(self, batch: Tuple[torch.Tensor]) -> torch.Tensor:
        # To be overridden by inherited classes.
        raise NotImplementedError

    def get_dataloader(self,
                       split: str,
                       batch_size: int,
                       shuffle: bool = False) -> DataLoader:
        # To be overridden by inherited classes.
        raise NotImplementedError

    @classmethod
    def add_model_specific_args(cls, parser: ArgumentParser) -> ArgumentParser:
        """
        Add arguments to the parser and return the parser.
        """
        # Required arguments:
        parser.add_argument('--vocab_filename',
                            default=None,
                            type=str,
                            required=False,  # changing to false since we're not using vocab
                            help="File name of the feature.")
        # Optional arguments:
        parser.add_argument('--optimizer',
                            default='adam',
                            type=str,
                            help="The optimizer to use, such as sgd or adam.")
        parser.add_argument('--learning_rate',
                            default=1e-3,
                            type=float,
                            help="The initial learning rate for training.")
        parser.add_argument('--l2_regularization',
                            default=False,
                            help="Whether to add L2 regularization.")
        parser.add_argument('--max_epochs',
                            default=10,
                            type=int,
                            help="The number of epochs to train your model.")
        parser.add_argument('--train_batch_size', default=32, type=int)
        parser.add_argument('--eval_batch_size', default=32, type=int)
        parser.add_argument('--seed',
                            type=int,
                            default=42,
                            help="The random seed for initialization")
        parser.add_argument('--do_train',
                            action="store_true",
                            default=True,
                            help="Whether to run training.")
        parser.add_argument('--do_predict',
                            action="store_true",
                            help="Whether to run predictions on the test set.")
        parser.add_argument('--data_dir',
                            default="data",
                            type=str,
                            help="The input data dir. Should contain the training files.")
        parser.add_argument('--output_dir',
                            type=str,
                            help=("The output directory where the model predictions "
                                  "and checkpoints will be written."))
        # NOTE: Set --gpus 0 or change the default value to 0 if not using GPUS.
        # See this [link](https://pytorch-lightning.readthedocs.io/en/latest/accelerators/gpu.html) for usage of this argument.
        parser.add_argument('--gpus',
                            default=1,
                            type=int,
                            help="The number of GPUs allocated for this, 0 meaning none")
        parser.add_argument('--num_workers',
                            default=8,
                            type=int,
                            help="Config `DataLoader` of pytorch")
        return parser


def generic_train(args: argparse.Namespace,
                  model_class: Type[pl.LightningModule]) -> Dict:
    """
        Train (and optionally predict) and return dict results.
        # Parameters
        args : `argparse.Namespace`, required.
            Configuration of the training and the model
        model_class : `Type[pl.LightningModule]`, required.
            Class of the model to be trained.
        # Returns
        A `dict` object containing the following keys and types.
            trainer: `pl.Trainer`
            model: `pl.LightningModule`
            val_results_best: `list[dict]`
                If `args.do_predict==True`
            test_results_best: `list[dict]`
                If `args.do_predict==True`
            best_model_path: `Path`
                Path to the checkpoint of the best model.
        """
    pl.seed_everything(args.seed)

    tensorboard_log_dir = Path(args.output_dir).joinpath('tensorboard_logs')
    tensorboard_log_dir.mkdir(parents=True, exist_ok=True)

    # Tensorboard logger
    tensorboard_logger = pl_loggers.TensorBoardLogger(
        save_dir=tensorboard_log_dir,
        version='version_' + datetime.now().strftime('%Y%m%d-%H%M%S'),
        name='',
        default_hp_metric=True)
    # Checkpoint callback
    checkpoint_dir = Path(args.output_dir).joinpath(tensorboard_logger.version,
                                                    'checkpoints')
    checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=checkpoint_dir,
                                                       filename='{epoch}-{val_mse:.2f}',
                                                       monitor='val_mse',
                                                       mode='min',   # minimize mse
                                                       save_top_k=1,
                                                       verbose=True)

    dict_args = vars(args)
    model = model_class(**dict_args)
    trainer = pl.Trainer.from_argparse_args(args,
                                            logger=tensorboard_logger,
                                            callbacks=[checkpoint_callback])

    output_dict = {'trainer': trainer, 'model': model}

    if args.do_train:
        trainer.fit(model=model)
        # Track model performance under differnt hparams settings in "Hparams" of TensorBoard
        tensorboard_logger.log_hyperparams(
            params=model.hparams,
            metrics={'hp_metric': checkpoint_callback.best_model_score.item()})
        tensorboard_logger.save()

        # Save the best model to `best_model.ckpt`
        best_model_path = checkpoint_dir.joinpath('best_model.ckpt')
        logger.info(f"Copy best model from {checkpoint_callback.best_model_path} "
                    f"to {best_model_path}.")
        shutil.copy(checkpoint_callback.best_model_path, best_model_path)

        output_dict.update({
            'trainer': trainer,
            'model': model,
            'best_model_path': best_model_path
        })

    # Optionally, predict on test set.
    if args.do_predict:
        best_model_path = checkpoint_dir.joinpath('best_model.ckpt')
        model = model.load_from_checkpoint(best_model_path)
        val_results_best = trainer.validate(model, verbose=True)
        test_results_best = trainer.test(model, verbose=True)
        print("Validation MSE on the best model: {: .4f}".format(
            val_results_best[0]['val_mse']))
        print("Validation R2 on the best model: {: .4f}".format(
            val_results_best[0]['val_r2']))
        #print('val_results_best all',val_results_best[0])
        #print('val_results_best_trulyall',val_results_best)
        print("Test MSE on the best model: {: .4f}".format(
            test_results_best[0]['test_mse']))
        print("Test R2 on the best model: {: .4f}".format(
            test_results_best[0]['test_r2']))
        output_dict.update({
            'val_results_best': val_results_best,
            'test_results_best': test_results_best,
        })

    return output_dict


In [15]:
class FeatureBasedRegressionModule(RegressionModule):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def get_model(self) -> nn.Module:
        return LinearRegressionModel(num_features=252) # use 764 for full embeddings, 252 for no bert

    def batch2input(self, batch):
        return {'features': batch[0]}

    def batch2labels(self, batch):
        return batch[1]

    def get_dataloader(self,
                       split: str,
                       batch_size: int,
                       shuffle: bool = False) -> DataLoader:
        # NOTE: In order to use different features, change feature_name by
        # passing `--feature_name <feature_name>` in the training loop in
        # the cell below, or revise the code here for correct paths if needed.
        data_dir = Path(self.hparams.data_dir)
        features_filepath = data_dir.joinpath(
            f"{split}_{self.hparams.feature_name}_features.npz")
        labels_filepath = data_dir.joinpath(f"{split}_{self.hparams.label_name}_labels.npz")
        features = sparse.load_npz(features_filepath).todense()
        print('features shape:',features.shape)
        #labels = np.load(labels_filepath, allow_pickle=True)["arr_0"]
        labels = np.asarray(sparse.load_npz(labels_filepath).todense()).ravel()
        print('labels shape:',labels.shape)
        dataset = torch.utils.data.TensorDataset(
            torch.from_numpy(features).float(),
            torch.from_numpy(labels).float())

        logger.info(f"Loading {split} features and labels "
                    f"from {features_filepath} and {labels_filepath}")
        data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                                  batch_size=batch_size,
                                                  shuffle=shuffle,
                                                  num_workers=self.hparams.num_workers)
        return data_loader

    @classmethod
    def add_model_specific_args(cls, parser: ArgumentParser) -> ArgumentParser:
        parser = super().add_model_specific_args(parser)
        # Required arguments:
        parser.add_argument('--feature_name',
                            default=None,
                            type=str,
                            required=True,
                            help="Name of the feature")
                            
        parser.add_argument('--label_name',
                            default=None,
                            type=str,
                            required=True,
                            help="Name of the label")
        # Optional arguments:
        parser.add_argument('--task',
                            default='featurebinarycls',
                            type=str,
                            help="Name of the task.")
        return parser


In [12]:
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Load hyperparameters
parser = ArgumentParser()
parser = FeatureBasedRegressionModule.add_model_specific_args(parser)

# NOTE: You should replace `unigram_binary` in the assignment statement of `args_str =...`
# with whatever feature that you are experimented with.
# You can also configure other options listed in the method of add_model_specific_args of
# the pytorch-lightning model `FeatureBasedBinaryClassificationLModule`.
args_str = ("--feature_name roberta --max_epochs 10 --label_name stay_len "
            "--output_dir output/sl_linear --optimizer adam --do_train --do_predict")

args = parser.parse_args(args_str.split())

# If output_dir not provided, a folder is generated
if args.output_dir is None:
    args.output_dir = str(
        Path('output').joinpath(
            f"{args.task}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"))
Path(args.output_dir).mkdir(parents=True, exist_ok=True)

print(f"Parsed arguments: {args}")

training_out_rob_10 = generic_train(args=args,
                                model_class=FeatureBasedRegressionModule)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Parsed arguments: Namespace(vocab_filename=None, optimizer='adam', learning_rate=0.001, l2_regularization=False, max_epochs=10, train_batch_size=32, eval_batch_size=32, seed=42, do_train=True, do_predict=True, data_dir='data', output_dir='output/sl_linear', gpus=1, num_workers=8, feature_name='roberta', label_name='stay_len', task='featurebinarycls')



  | Name  | Type                        | Params
------------------------------------------------------
0 | model | LinearRegressionModel       | 765   
1 | mse   | MeanSquaredError            | 0     
2 | mape  | MeanAbsolutePercentageError | 0     
3 | mae   | MeanAbsoluteError           | 0     
4 | r2    | R2Score                     | 0     
------------------------------------------------------
765       Trainable params
0         Non-trainable params
765       Total params
0.003     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

2022-03-15 10:17:17 - INFO - __main__ - Loading dev features and labels from data/dev_roberta_features.npz and data/dev_stay_len_labels.npz


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_loss': 1128509734912.0,
 'val_mae': 599885.6875,
 'val_mape': 2.469804048538208,
 'val_mse': 1128509734912.0,
 'val_r2': 2.2260525226593018}
--------------------------------------------------------------------------------


2022-03-15 10:19:56 - INFO - __main__ - Loading test features and labels from data/test_roberta_features.npz and data/test_stay_len_labels.npz


features shape: (11473, 764)
labels shape: (11473,)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1065926262784.0,
 'test_mae': 605325.5625,
 'test_mape': 2.2779018878936768,
 'test_mse': 1065926262784.0,
 'test_r2': 2.16449236869812}
--------------------------------------------------------------------------------
Validation MSE on the best model:  1128509734912.0000
Validation R2 on the best model:  2.2261
Test MSE on the best model:  1065926262784.0000
Test R2 on the best model:  2.1645


In [13]:
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Load hyperparameters
parser = ArgumentParser()
parser = FeatureBasedRegressionModule.add_model_specific_args(parser)

# NOTE: You should replace `unigram_binary` in the assignment statement of `args_str =...`
# with whatever feature that you are experimented with.
# You can also configure other options listed in the method of add_model_specific_args of
# the pytorch-lightning model `FeatureBasedBinaryClassificationLModule`.
args_str = ("--feature_name roberta --max_epochs 20 --label_name stay_len "
            "--output_dir output/sl_linear --optimizer adam --do_train --do_predict "
            )

args = parser.parse_args(args_str.split())

# If output_dir not provided, a folder is generated
if args.output_dir is None:
    args.output_dir = str(
        Path('output').joinpath(
            f"{args.task}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"))
Path(args.output_dir).mkdir(parents=True, exist_ok=True)

print(f"Parsed arguments: {args}")

training_out_rob_20 = generic_train(args=args,
                                model_class=FeatureBasedRegressionModule)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | LinearRegressionModel       | 765   
1 | mse   | MeanSquaredError            | 0     
2 | mape  | MeanAbsolutePercentageError | 0     
3 | mae   | MeanAbsoluteError           | 0     
4 | r2    | R2Score                     | 0     
------------------------------------------------------
765       Trainable params
0         Non-trainable params
765       Total params
0.003     Total estimated model params size (MB)


Parsed arguments: Namespace(vocab_filename=None, optimizer='adam', learning_rate=0.001, l2_regularization=False, max_epochs=20, train_batch_size=32, eval_batch_size=32, seed=42, do_train=True, do_predict=True, data_dir='data', output_dir='output/sl_linear', gpus=1, num_workers=8, feature_name='roberta', label_name='stay_len', task='featurebinarycls')


Validation sanity check: 0it [00:00, ?it/s]

2022-03-15 10:22:10 - INFO - __main__ - Loading dev features and labels from data/dev_roberta_features.npz and data/dev_stay_len_labels.npz


features shape: (11473, 764)
labels shape: (11473,)


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_loss': 1124531044352.0,
 'val_mae': 602737.5625,
 'val_mape': 2.5077879428863525,
 'val_mse': 1124531044352.0,
 'val_r2': 2.226785898208618}
--------------------------------------------------------------------------------


2022-03-15 10:27:22 - INFO - __main__ - Loading test features and labels from data/test_roberta_features.npz and data/test_stay_len_labels.npz


features shape: (11473, 764)
labels shape: (11473,)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1064722104320.0,
 'test_mae': 607866.5625,
 'test_mape': 2.314664125442505,
 'test_mse': 1064722104320.0,
 'test_r2': 2.167832612991333}
--------------------------------------------------------------------------------
Validation MSE on the best model:  1124531044352.0000
Validation R2 on the best model:  2.2268
Test MSE on the best model:  1064722104320.0000
Test R2 on the best model:  2.1678


In [14]:
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Load hyperparameters
parser = ArgumentParser()
parser = FeatureBasedRegressionModule.add_model_specific_args(parser)

# NOTE: You should replace `unigram_binary` in the assignment statement of `args_str =...`
# with whatever feature that you are experimented with.
# You can also configure other options listed in the method of add_model_specific_args of
# the pytorch-lightning model `FeatureBasedBinaryClassificationLModule`.
args_str = ("--feature_name roberta --max_epochs 50 --label_name stay_len "
            "--output_dir output/sl_linear --optimizer adam --do_train --do_predict "
            )

args = parser.parse_args(args_str.split())

# If output_dir not provided, a folder is generated
if args.output_dir is None:
    args.output_dir = str(
        Path('output').joinpath(
            f"{args.task}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"))
Path(args.output_dir).mkdir(parents=True, exist_ok=True)

print(f"Parsed arguments: {args}")

training_out_rob_50 = generic_train(args=args,
                                model_class=FeatureBasedRegressionModule)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | LinearRegressionModel       | 765   
1 | mse   | MeanSquaredError            | 0     
2 | mape  | MeanAbsolutePercentageError | 0     
3 | mae   | MeanAbsoluteError           | 0     
4 | r2    | R2Score                     | 0     
------------------------------------------------------
765       Trainable params
0         Non-trainable params
765       Total params
0.003     Total estimated model params size (MB)


Parsed arguments: Namespace(vocab_filename=None, optimizer='adam', learning_rate=0.001, l2_regularization=False, max_epochs=50, train_batch_size=32, eval_batch_size=32, seed=42, do_train=True, do_predict=True, data_dir='data', output_dir='output/sl_linear', gpus=1, num_workers=8, feature_name='roberta', label_name='stay_len', task='featurebinarycls')


Validation sanity check: 0it [00:00, ?it/s]

2022-03-15 10:37:46 - INFO - __main__ - Loading dev features and labels from data/dev_roberta_features.npz and data/dev_stay_len_labels.npz


features shape: (11473, 764)
labels shape: (11473,)


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_loss': 1124215029760.0,
 'val_mae': 601289.875,
 'val_mape': 2.4931983947753906,
 'val_mse': 1124215029760.0,
 'val_r2': 2.2271676063537598}
--------------------------------------------------------------------------------


2022-03-15 10:50:50 - INFO - __main__ - Loading test features and labels from data/test_roberta_features.npz and data/test_stay_len_labels.npz


features shape: (11473, 764)
labels shape: (11473,)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1065554214912.0,
 'test_mae': 606180.3125,
 'test_mape': 2.3022756576538086,
 'test_mse': 1065554214912.0,
 'test_r2': 2.169055223464966}
--------------------------------------------------------------------------------
Validation MSE on the best model:  1124215029760.0000
Validation R2 on the best model:  2.2272
Test MSE on the best model:  1065554214912.0000
Test R2 on the best model:  2.1691


BioClinicalBERT, truncated to 512

In [11]:
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Load hyperparameters
parser = ArgumentParser()
parser = FeatureBasedRegressionModule.add_model_specific_args(parser)

# NOTE: You should replace `unigram_binary` in the assignment statement of `args_str =...`
# with whatever feature that you are experimented with.
# You can also configure other options listed in the method of add_model_specific_args of
# the pytorch-lightning model `FeatureBasedBinaryClassificationLModule`.
args_str = ("--feature_name bc_512 --max_epochs 20 --label_name stay_len "
            "--output_dir output/sl_linear_bc --optimizer adam --do_train --do_predict "
            )

args = parser.parse_args(args_str.split())

# If output_dir not provided, a folder is generated
if args.output_dir is None:
    args.output_dir = str(
        Path('output').joinpath(
            f"{args.task}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"))
Path(args.output_dir).mkdir(parents=True, exist_ok=True)

print(f"Parsed arguments: {args}")

training_out_bc_20 = generic_train(args=args,
                                model_class=FeatureBasedRegressionModule)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Parsed arguments: Namespace(vocab_filename=None, optimizer='adam', learning_rate=0.001, l2_regularization=False, max_epochs=20, train_batch_size=32, eval_batch_size=32, seed=42, do_train=True, do_predict=True, data_dir='data', output_dir='output/sl_linear_bc', gpus=1, num_workers=8, feature_name='bc_512', label_name='stay_len', task='featurebinarycls')



  | Name  | Type                        | Params
------------------------------------------------------
0 | model | LinearRegressionModel       | 765   
1 | mse   | MeanSquaredError            | 0     
2 | mape  | MeanAbsolutePercentageError | 0     
3 | mae   | MeanAbsoluteError           | 0     
4 | r2    | R2Score                     | 0     
------------------------------------------------------
765       Trainable params
0         Non-trainable params
765       Total params
0.003     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

2022-03-15 10:06:37 - INFO - __main__ - Loading dev features and labels from data/dev_bc_512_features.npz and data/dev_stay_len_labels.npz


...

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_loss': 1150314348544.0,
 'val_mae': 607641.625,
 'val_mape': 2.4327030181884766,
 'val_mse': 1150314348544.0,
 'val_r2': 2.2510368824005127}
--------------------------------------------------------------------------------


2022-03-15 10:15:13 - INFO - __main__ - Loading test features and labels from data/test_bc_512_features.npz and data/test_stay_len_labels.npz


features shape: (11473, 764)
labels shape: (11473,)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1082034028544.0,
 'test_mae': 611502.8125,
 'test_mape': 2.2351739406585693,
 'test_mse': 1082034028544.0,
 'test_r2': 2.181267738342285}
--------------------------------------------------------------------------------
Validation MSE on the best model:  1150314348544.0000
Validation R2 on the best model:  2.2510
Test MSE on the best model:  1082034028544.0000
Test R2 on the best model:  2.1813


#### Only structured data, no BERT-derived embeddings

In [16]:
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Load hyperparameters
parser = ArgumentParser()
parser = FeatureBasedRegressionModule.add_model_specific_args(parser)

# NOTE: You should replace `unigram_binary` in the assignment statement of `args_str =...`
# with whatever feature that you are experimented with.
# You can also configure other options listed in the method of add_model_specific_args of
# the pytorch-lightning model `FeatureBasedBinaryClassificationLModule`.
args_str = ("--feature_name nobert --max_epochs 20 --label_name stay_len "
            "--output_dir output/sl_linear_nobert --optimizer adam --do_train --do_predict "
            )

args = parser.parse_args(args_str.split())

# If output_dir not provided, a folder is generated
if args.output_dir is None:
    args.output_dir = str(
        Path('output').joinpath(
            f"{args.task}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"))
Path(args.output_dir).mkdir(parents=True, exist_ok=True)

print(f"Parsed arguments: {args}")

training_out_nobert_20 = generic_train(args=args,
                                model_class=FeatureBasedRegressionModule)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | LinearRegressionModel       | 253   
1 | mse   | MeanSquaredError            | 0     
2 | mape  | MeanAbsolutePercentageError | 0     
3 | mae   | MeanAbsoluteError           | 0     
4 | r2    | R2Score                     | 0     
------------------------------------------------------
253       Trainable params
0         Non-trainable params
253       Total params
0.001     Total estimated model params size (MB)


Parsed arguments: Namespace(vocab_filename=None, optimizer='adam', learning_rate=0.001, l2_regularization=False, max_epochs=20, train_batch_size=32, eval_batch_size=32, seed=42, do_train=True, do_predict=True, data_dir='data', output_dir='output/sl_linear_nobert', gpus=1, num_workers=8, feature_name='nobert', label_name='stay_len', task='featurebinarycls')


Validation sanity check: 0it [00:00, ?it/s]

2022-03-15 11:00:53 - INFO - __main__ - Loading dev features and labels from data/dev_nobert_features.npz and data/dev_stay_len_labels.npz


features shape: (11473, 252)
labels shape: (11473,)


...

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2022-03-15 11:06:13 - INFO - __main__ - Loading test features and labels from data/test_nobert_features.npz and data/test_stay_len_labels.npz


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_loss': 1223072940032.0,
 'val_mae': 630249.625,
 'val_mape': 2.4313151836395264,
 'val_mse': 1223072940032.0,
 'val_r2': 2.328740119934082}
--------------------------------------------------------------------------------
features shape: (11473, 252)
labels shape: (11473,)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1151872270336.0,
 'test_mae': 634577.75,
 'test_mape': 2.230215549468994,
 'test_mse': 1151872270336.0,
 'test_r2': 2.2515149116516113}
--------------------------------------------------------------------------------
Validation MSE on the best model:  1223072940032.0000
Validation R2 on the best model:  2.3287
Test MSE on the best model:  1151872270336.0000
Test R2 on the best model:  2.2515


In [17]:
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO)
logger = logging.getLogger(__name__)

# Load hyperparameters
parser = ArgumentParser()
parser = FeatureBasedRegressionModule.add_model_specific_args(parser)

# NOTE: You should replace `unigram_binary` in the assignment statement of `args_str =...`
# with whatever feature that you are experimented with.
# You can also configure other options listed in the method of add_model_specific_args of
# the pytorch-lightning model `FeatureBasedBinaryClassificationLModule`.
args_str = ("--feature_name nobert --max_epochs 20 --label_name stay_len "
            "--output_dir output/sl_linear_nobert --optimizer adam --do_train --do_predict "
            "--l2_regularization True")

args = parser.parse_args(args_str.split())

# If output_dir not provided, a folder is generated
if args.output_dir is None:
    args.output_dir = str(
        Path('output').joinpath(
            f"{args.task}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"))
Path(args.output_dir).mkdir(parents=True, exist_ok=True)

print(f"Parsed arguments: {args}")

training_out_nobert_20_l2 = generic_train(args=args,
                                model_class=FeatureBasedRegressionModule)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Parsed arguments: Namespace(vocab_filename=None, optimizer='adam', learning_rate=0.001, l2_regularization='True', max_epochs=20, train_batch_size=32, eval_batch_size=32, seed=42, do_train=True, do_predict=True, data_dir='data', output_dir='output/sl_linear_nobert', gpus=1, num_workers=8, feature_name='nobert', label_name='stay_len', task='featurebinarycls')


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | LinearRegressionModel       | 253   
1 | mse   | MeanSquaredError            | 0     
2 | mape  | MeanAbsolutePercentageError | 0     
3 | mae   | MeanAbsoluteError           | 0     
4 | r2    | R2Score                     | 0     
------------------------------------------------------
253       Trainable params
0         Non-trainable params
253       Total params
0.001     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

2022-03-15 12:38:25 - INFO - __main__ - Loading dev features and labels from data/dev_nobert_features.npz and data/dev_stay_len_labels.npz


...

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2022-03-15 12:43:17 - INFO - __main__ - Loading test features and labels from data/test_nobert_features.npz and data/test_stay_len_labels.npz


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'val_loss': 1223072940032.0,
 'val_mae': 630249.625,
 'val_mape': 2.4313151836395264,
 'val_mse': 1223072940032.0,
 'val_r2': 2.328740119934082}
--------------------------------------------------------------------------------
features shape: (11473, 252)
labels shape: (11473,)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1151872270336.0,
 'test_mae': 634577.75,
 'test_mape': 2.230215549468994,
 'test_mse': 1151872270336.0,
 'test_r2': 2.2515149116516113}
--------------------------------------------------------------------------------
Validation MSE on the best model:  1223072940032.0000
Validation R2 on the best model:  2.3287
Test MSE on the best model:  1151872270336.0000
Test R2 on the best model:  2.2515


### References

[Linear Regression with PyTorch](https://towardsdatascience.com/linear-regression-with-pytorch-eb6dedead817)  
